In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| output: false
import numpy as np
import tempfile
from pandas import DataFrame
from pathlib import Path
import pandas as pd

from spannerlib.utils import checkLogs
from spannerlib.span import Span
from spannerlib.session import Session,test_session

In [ ]:
test_session(
    ["""
    new S(int,int)
    new S2(int,int)
    R(X,Y)<-S(X,Y)
    R(X,Y)<-S(X,Z),R(Z,Y)
    R2(X,Y,Z)<-S2(X,Y),R(Y,Z)
    R2(X,Y,Z)<-S2(X,Y),R(X,Y),R2(X,Y,Y),S(X,Z)
    """,
    """?R2(X,Y,Z)"""
    ],
    [
        None,
        pd.DataFrame(columns=['X','Y','Z'])
    ],
    debug=True
)

?R2(X,Y,Z)


In [ ]:
# test_session(
#     [
#         """
#         test_range(X) <- yield_range(5) -> (X)
#         ?test_range(X)
#         """,
#         """
#         test_range_span(X) <- yield_range_span(5) -> (X)
#         ?test_range_span(X)
#         """,
#         """
#         test_range_str(X) <- yield_range_str(5) -> (X)
#         ?test_range_str(X)
#         """,
#     ],
#     [
#         pd.DataFrame({'X':range(5)}),
#         pd.DataFrame({'X':[Span(0,0),Span(0,1),Span(0,2),Span(0,3),Span(0,4)]}),
#         pd.DataFrame({'X':["string0","string1","string2","string3","string4"]}),
#     ],

# )


In [ ]:
# with checkLogs(name='spannerlib.engine'):
test_session(
    [
        """
            new parent(str, str)
            parent("Liam", "Noah")
            parent("Noah", "Oliver")
            parent("James", "Lucas")
            parent("Noah", "Benjamin")
            parent("Benjamin", "Mason")
            ancestor(X,Y) <- parent(X,Y)
            ancestor(X,Y) <- parent(X,Z), ancestor(Z,Y)
        """,
        """
            ?ancestor("Liam", X)
        """

    ],
    [   None,
        pd.DataFrame({'X':['Mason','Oliver','Benjamin','Noah']}),
    ],
    # debug=True
)


'?ancestor("Liam",X)'

,X
0,Noah
5,Oliver
6,Benjamin
7,Mason


In [ ]:
test_session(
    [
        """
            new parent(str, str)
            parent("Liam", "Noah")
            parent("Noah", "Oliver")
            parent("James", "Lucas")
            parent("Noah", "Benjamin")
            parent("Benjamin", "Mason")
            ancestor(X,Y) <- parent(X,Y)
            ancestor(X,Y) <- parent(X,Z), ancestor(Z,Y)

            ?ancestor("Liam", X)
        """,
        """
            ?ancestor(X, "Mason")

        """,
        """
            ?ancestor("Mason", X)

        """,
    ],
    [
        pd.DataFrame({'X':['Mason','Oliver','Benjamin','Noah']}),
        pd.DataFrame({'X':['Noah','Liam','Benjamin']}),
        pd.DataFrame({'X':[]}),
    ],

)


'?ancestor("Liam",X)'

,X
0,Noah
5,Oliver
6,Benjamin
7,Mason


'?ancestor(X,"Mason")'

,X
4,Benjamin
7,Liam
8,Noah


'?ancestor("Mason",X)'

,X


In [ ]:
def test_mutually_recursive_basic() -> None:
    commands = """
            new C(int)
            C(1)
            C(2)
            C(3)

            B(X) <- C(X)
            A(X) <- B(X)
            B(X) <- A(X)

            ?A(X)
            """

    expected_result = f"""{QUERY_RESULT_PREFIX}'A(X)':
        X
        -----
        1
        2
        3
    """

    run_test(commands, expected_result)

test_mutually_recursive_basic()

printing results for query 'A(X)':
   X
-----
   1
   2
   3

